In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzle 1 - Ejemplo.
Is it possible for any inhabitant of this island to claim that he is a knave?


No; no inhabitant can claim to be a knave because no knight would lie and say he is a knave and no knave would truthfully admit to being a knave.

In [5]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): ¬[x.Says[x.Knave]] = 
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [6]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing =
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [7]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = P.P1(x)
    def knightsAreTruthTellers[P](x: Inhabitant) = P.P2[P](x)
    def knavesAreLiers[P](x: Inhabitant) = P.P3[P](x)
    
    def noKnightLies[P](x: Inhabitant): x.Knight => x.Says[Not[P]] => P => False = 
        xIsKnight => xSaysNotP => p => 
            P.P2[Not[P]](x)(xIsKnight)(xSaysNotP)(p)
    
    def noKnaveTellsTruth[P](x: Inhabitant): x.Knave => x.Says[P] => P => False = 
        xIsKnave => xSaysP => p => 
            P.P3[P](x)(xIsKnave)(xSaysP)(p)
}

defined class SomeSugar

In [8]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬ Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => 
                // no knight would lie
                premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave)(xIsKnight),
            xIsKnave => 
                // no knave would tell the truth
                premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave)(xIsKnave)
        )

defined function puzzle1

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 4. Suppose A instead says: "Exactly one of us is a knave." What can be deduced about A and what can be deduced about B?

A is saying that exactly one of the persons A and B is a knave. If A is a knight, his statement is true, exactly one is a knave, and so B is a knave. If A is a knave, his statement is false, hence B must again be a knave, because if B were a knight, then it would be true that exactly one is a knave! And
so regardless of whether A is a knight or a knave, B is a knave.
As for A, his type cannot be determined; he could be either a knight or a knave.

In [9]:
def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = ???
    /* Exactamente uno de los dos es Knave (o uno u otro):
    1.- X es Knave
    2.- Y es Knave
    
    */

defined function puzzle4

In [10]:
def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = ???
    

defined function puzzle4

In [11]:
// VERSION 1

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_4R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] =>
                    (x_4R match{
                        case Left(_) => 
                            (Left(xKnave), x_3R(x_4R)) : (Either[x.Knave, x.Knight],y.Knave)
                            // O devolveria un Nothing?:
                            // x_3R(x_4R) : Nothing
                        case Right(_) => 
                            (Left(xKnave), x_3R(x_4R)) : (Either[x.Knave, x.Knight],y.Knave)
                            // O devolveria un Nothing?:
                            // x_3R(x_4R) : Nothing
                    }) : (Either[x.Knave, x.Knight],y.Knave) // O devolveria un Nothing?
            
                (Left(xKnave), x_3R(x_4R)) : (Either[x.Knave, x.Knight],y.Knave)
                // ¿Porqué teniendo ya algo de tipo (Either[x.Knave, x.Knight],y.Knave) no puedo quitar las ultimas ??? ?
        })
    
    ??? : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [12]:
//SIMPLIFICADO VERSION 1
def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    (premises.P1(x) match{
        case Left(xKnight) =>
            xsayL => 
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xsayL)
                
                (x_3L match{
                    case Left(x_3L_left) => 
                        val x_3L_left_contradiccion: x.Knight => Nothing = x_3L_left._1
                        (Left(x_3L_left._1), x_3L_left_contradiccion(xKnight))
                    case Right(x_3L_right) => 
                        (Right(x_3L_right._1), x_3L_right._2)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
        
        case Right(xKnave) =>
            xsayR => 
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Not[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Not[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xsayR) 
                                           
                {x_4R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => 
                    (x_4R match{
                        case Left(x_4R_left) => 
                            x_3R(x_4R) : Nothing
                        case Right(x_4R_right) => 
                            val x_4R_right_contradiccion: x.Knight => Nothing = xKnave
                            x_4R_right_contradiccion(x_4R_right._1): Nothing
                            
                    }) : Nothing}
            // No puedo sacar este valor. ¿Donde esta el error?
            ??? : (Either[x.Knave, x.Knight],y.Knave)
    }): (x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave))
    

defined function puzzle4

In [13]:
// VERSION 2

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_3R(Left(xKnave: x.Knave, ??? : y.Knight)): Nothing
            
            ??? : (Either[x.Knave, x.Knight],y.Knave)
                
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [14]:
// VERSION 2

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    ??? : y.Knight
                                                case Right(yKnave: y.Knave) =>
                                                    ??? : y.Knight
                                            }) : y.Knight)): Nothing
            
            ??? : (Either[x.Knave, x.Knight],y.Knave)
                
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [15]:
// VERSION 2

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    yKnight : y.Knight
                                            }) : y.Knight)): Nothing
            
            (Left(xKnave): Either[x.Knave, x.Knight], ??? : y.Knave) 
                
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [16]:
// VERSION 2

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    yKnight : y.Knight
                                            }) : y.Knight)): Nothing
            
                (Left(xKnave): Either[x.Knave, x.Knight], x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    yKnight : y.Knight
                                            }) : y.Knight)) : y.Knave) 
                
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [17]:
// VERSION 2

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    yKnight : y.Knight
                                            }) : y.Knight)): Nothing
            
                (Left(xKnave): Either[x.Knave, x.Knight], x_3R(Left(xKnave: x.Knave, (premises.P1(y) match{
                                                case Left(yKnight: y.Knight) =>
                                                    yKnight : y.Knight
                                            }) : y.Knight)) : y.Knave) 
                
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [19]:
// VERSION 2 - SIMPLIFICADO

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay =>
        (premises.P1(x) match{
            case Left(xKnight) => 
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1), x_3L_left_not(xKnight))
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1), x_3L_right._2)
               
                })
            
            case Right(xKnave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
            
                (Left(xKnave), x_3R(Left(xKnave, (premises.P1(y) match{
                                                case Left(yKnight) =>
                                                    yKnight 
                                            }) 
                                        )) 
                ) 
                
        }) 
    })
    

defined function puzzle4

In [18]:
// VERSION 3

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
                
            
                // En el pattern matching hay que expresar la rama Left y Right (siempre ambas)
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => x_3R(Left(xKnave, yKnight)) : (Either[x.Knave, x.Knight],y.Knave)
                    // En este caso si a x_3R le paso un (x.Knave,y.Knight) obtengo un Nothing
                    // El tipo Nothing es equivalente a cualquier otro tipo.
                    // En este caso si tengo un Nothing tengo un (Either[x.Knave, x.Knight],y.Knave)
                    case Right(yKnave: y.Knave) => (Left(xKnave), yKnave) : (Either[x.Knave, x.Knight],y.Knave)
                }
            
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [33]:
// VERSION 3

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] =>
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
            /*
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        //Funcionan ambas versiones
                        //(Right(xKnight), x_3L_left_contradiccion(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                        (Left(x_3L_left._1):Either[x.Knave, x.Knight], x_3L_left_not(xKnight) : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1):Either[x.Knave, x.Knight], x_3L_right._2 : y.Knave) : (Either[x.Knave, x.Knight],y.Knave)
               
                }) : (Either[x.Knave, x.Knight],y.Knave)
            */
            // Otra opcion de sacar la solucion (Either[x.Knave, x.Knight],y.Knave)
            // Que al final es lo mismo pero mas simplificado haciendo uso tambien de "type Knave = ¬[Knight]" pero sin "val"
                
                (x_3L match{
                    case Left(x_3L_left: (x.Knave,y.Knight)) => 
                        x_3L_left._1(xKnight) : (Either[x.Knave, x.Knight],y.Knave)
                        // Si al x.Knave de la tupla x_3L_left le paso un x.Knight obtengo un Nothing.
                        // Esto sale de type Knave = ¬[Knight]
                        // Cualquier tipo de datos es Nothing, por tanto Nothing es tambien de tipo (Either[x.Knave, x.Knight],y.Knave)
                    case Right(x_3L_right: (x.Knight,y.Knave)) => 
                        (Right(x_3L_right._1), x_3L_right._2) : (Either[x.Knave, x.Knight],y.Knave)
                        // De la tupla x_3L_right me saco ambos argumentos de tipo (Either[x.Knave, x.Knight],y.Knave)
                }) : (Either[x.Knave, x.Knight],y.Knave) // Este tipo de datos se puede obviar y quitar tambien los () del match
                
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
                
            
                // En el pattern matching hay que expresar la rama Left y Right (siempre ambas)
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => x_3R(Left(xKnave, yKnight)) : (Either[x.Knave, x.Knight],y.Knave)
                    // En este caso si a x_3R le paso un (x.Knave,y.Knight) obtengo un Nothing
                    // El tipo Nothing es equivalente a cualquier otro tipo.
                    // En este caso si tengo un Nothing tengo un (Either[x.Knave, x.Knight],y.Knave)
                    case Right(yKnave: y.Knave) => (Left(xKnave), yKnave) : (Either[x.Knave, x.Knight],y.Knave)
                }
            
        }) : (Either[x.Knave, x.Knight],y.Knave)})
    

defined function puzzle4

In [23]:
// VERSION 3 - SIMPLIFICADO

def puzzle4(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knave) = 
    ({xSay =>
        (premises.P1(x) match{
            case Left(xKnight) => 
                val x_1L: x.Knight => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = premises.P2[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                
                val x_2L: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left) => 
                        val x_3L_left_not: x.Knight => Nothing = x_3L_left._1
                        (Left(x_3L_left._1), x_3L_left_not(xKnight)) 
                    case Right(x_3L_right) => 
                        (Right(x_3L_right._1), x_3L_right._2) 
                })
            
            case Right(xKnave) => 
                val x_1R: x.Knave => 
                            x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = premises.P3[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] => 
                                ¬[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)] => Nothing = x_2R(xSay)
                
                premises.P1(y) match{
                    case Left(yKnight) => x_3R(Left(xKnave, yKnight)) 
                    case Right(yKnave) => (Left(xKnave), yKnave) 
                }
        }) 
    })
    

defined function puzzle4